In [42]:
import os
import sys
import json
import torch
import importlib
import torch.nn as nn
import pandas as pd
from typing import Dict, List, Any
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
# Get the current working directory
scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)


import src.deep_unfolding_nn
importlib.reload(src.deep_unfolding_nn)
from src.deep_unfolding_nn import DUNN, Trainer

import src.utils
importlib.reload(src.utils)
from src.utils import sum_rate_loss_BC

In [43]:
def proj_power(V, PT_sc):
    num_users_sc = len(V)
    # Projects V according to the constraint
    alph = torch.sqrt(torch.tensor(PT_sc)) / torch.sqrt(torch.tensor(sum([torch.trace(V[str(k)] @ V[str(k)].conj().T) for k in range(num_users_sc)])))
    V_proj = {str(k): alph * V[str(k)] for k in range(num_users_sc)}
    return V_proj

def init_V(H):
    # Initializes V according to Hu's code
    V = {}
    for k in range(len(H_dict)):
        V[str(k)] = (torch.linalg.pinv(H[str(k)] @ H[str(k)].conj().T) @ H[str(k)]).conj().T
    return V

In [44]:
class setup():
    def __init__(self, n_tx, n_rx, d, num_streams, num_users, PT, sig):
        self.n_tx = n_tx
        self.n_rx = n_rx
        self.d = num_streams
        self.K = num_users
        self.PT = PT
        self.sig = sig

In [45]:
set_up = setup(n_tx=8, n_rx=[2, 2, 2, 2], d=[2, 2, 2, 2], num_streams=[2, 2, 2, 2], num_users=4, PT=5, sig = [.1, .1, .1, .1])
# Parameters
n_t = set_up.n_tx  # fixed number of transmit antennas
n_r = set_up.n_rx  # different receive antennas for each column

# Function to generate one complex matrix
def complex_matrix(n_t, n_r):
    return torch.randn(n_r, n_t, dtype=torch.cfloat)

# Construct the DataFrame
data = []
for _ in range(10):  # 5 rows
    row = {f'user_{i}': complex_matrix(n_t, n_r[i]) for i in range(set_up.K)}
    data.append(row)

H = pd.DataFrame(data)

In [48]:
V_init_dict = {}

for idx, row in H.iterrows():
    row = H.iloc[0]  # Get first row
    H_dict = {str(i): row[i] for i in range(len(row))}
    V_init = init_V(H_dict)
    V_init_proj = proj_power(V_init, set_up.PT)
    V_init_dict[str(idx)] = V_init_proj

/var/folders/35/f3dbnk6d3ts1993_z878dt5w0000gn/T/ipykernel_29652/2841192314.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  H_dict = {str(i): row[i] for i in range(len(row))}
/var/folders/35/f3dbnk6d3ts1993_z878dt5w0000gn/T/ipykernel_29652/4217901526.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  alph = torch.sqrt(torch.tensor(PT_sc)) / torch.sqrt(torch.tensor(sum([torch.trace(V[str(k)] @ V[str(k)].conj().T) for k in range(num_users_sc)])))


In [ ]:
# V = {}
# d = set_up.d
# for i in range(10):
#     V[str(i)] = {}
#     for j in range(set_up.K):
#         V[str(i)][str(j)] = torch.randn(n_t, d[j], dtype=torch.cfloat)

In [49]:
# set_up = setup(n_tx=8, n_rx=[2, 2, 2, 2], num_streams=[2, 2, 2, 2], num_users=4, P=5)
du = DUNN(num_layers=5, setup=set_up)

tr = Trainer(model=du, setup=set_up, lr=1e-3)
tr.train_epoch(H, loss_fn=sum_rate_loss_BC, num_epochs=10, batch_size=2, V_init = V_init_dict)

nan